In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/book-recommendation-dataset/Ratings.csv
/kaggle/input/book-recommendation-dataset/Users.csv
/kaggle/input/book-recommendation-dataset/classicRec.png
/kaggle/input/book-recommendation-dataset/Books.csv
/kaggle/input/book-recommendation-dataset/DeepRec.png
/kaggle/input/book-recommendation-dataset/recsys_taxonomy2.png


In [2]:
os.getcwdb()

b'/kaggle/working'

### Content based Recommendation system

In [3]:
import matplotlib as plt
import seaborn as sns

In [4]:
rating= pd.read_csv(r'/kaggle/input/book-recommendation-dataset/Ratings.csv')

In [5]:
rating.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
users= pd.read_csv(r'/kaggle/input/book-recommendation-dataset/Users.csv')
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [7]:
books= pd.read_csv(r'/kaggle/input/book-recommendation-dataset/Books.csv')
books.head()

/tmp/ipykernel_17/2189789844.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books= pd.read_csv(r'/kaggle/input/book-recommendation-dataset/Books.csv')


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [8]:
print(f'books count- {books.shape}')
print(f'users count-{users.shape}')
print(f'ratings count- {rating.shape}')

books count- (271360, 8)
users count-(278858, 3)
ratings count- (1149780, 3)


In [9]:
books.isna().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [10]:
users.isna().sum() # total count- 278858

User-ID          0
Location         0
Age         110762
dtype: int64

In [11]:
rating.isna().sum() # total count- 1149780

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

#### Pre-processing

In [12]:
print(f' duplicate books- {books.duplicated().sum()}')
print(f' duplicate users- {users.duplicated().sum()}')
print(f' duplicate rating- {rating.duplicated().sum()}')

 duplicate books- 0
 duplicate users- 0
 duplicate rating- 0


### EDA

In [13]:
books.describe()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
count,271360,271360,271358,271360,271358,271360,271360,271357
unique,271360,242135,102022,202,16807,271044,271044,271041
top,0195153448,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/225307649X.0...
freq,1,27,632,13903,7535,2,2,2


In [14]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [15]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [16]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [17]:
user_rating = pd.merge(users, rating, on='User-ID', how='inner')

In [18]:
user_rating.head()

,User-ID,Location,Age,ISBN,Book-Rating
0,2,"stockton, california, usa",18.0,0195153448,0
1,7,"washington, dc, usa",NaN,034542252,0
2,8,"timmins, ontario, canada",NaN,0002005018,5
3,8,"timmins, ontario, canada",NaN,0060973129,0
4,8,"timmins, ontario, canada",NaN,0374157065,0


In [19]:
user_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 5 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   User-ID      1149780 non-null  int64  
 1   Location     1149780 non-null  object 
 2   Age          840288 non-null   float64
 3   ISBN         1149780 non-null  object 
 4   Book-Rating  1149780 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 43.9+ MB


In [20]:
df = pd.merge(user_rating, books, on='ISBN', how='inner')

In [21]:
df.head()

,User-ID,Location,Age,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,2,"stockton, california, usa",18.0,0195153448,0,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,8,"timmins, ontario, canada",NaN,0002005018,5,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,8,"timmins, ontario, canada",NaN,0060973129,0,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,8,"timmins, ontario, canada",NaN,0374157065,0,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,8,"timmins, ontario, canada",NaN,0393045218,0,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031136 entries, 0 to 1031135
Data columns (total 12 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   User-ID              1031136 non-null  int64  
 1   Location             1031136 non-null  object 
 2   Age                  753301 non-null   float64
 3   ISBN                 1031136 non-null  object 
 4   Book-Rating          1031136 non-null  int64  
 5   Book-Title           1031136 non-null  object 
 6   Book-Author          1031134 non-null  object 
 7   Year-Of-Publication  1031136 non-null  object 
 8   Publisher            1031134 non-null  object 
 9   Image-URL-S          1031136 non-null  object 
 10  Image-URL-M          1031136 non-null  object 
 11  Image-URL-L          1031132 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 94.4+ MB


In [23]:
df = df.drop_duplicates()

In [24]:
df.describe(include='object')

,Location,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
count,1031136,1031136,1031136,1031134,1031136,1031134,1031136,1031136,1031132
unique,22480,270151,241071,101587,202,16729,269842,269842,269839
top,"toronto, ontario, canada",0971880107,Wild Animus,Stephen King,2002,Ballantine Books,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...
freq,14782,2502,2502,10053,87276,34724,2502,2502,2502


### Show Books with highest average rating

In [25]:
# number of votes on each book
df.groupby('Book-Title').count()

,User-ID,Location,Age,ISBN,Book-Rating,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
Book-Title,,,,,,,,,,,
"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",4,4,1,4,4,4,4,4,4,4,4
Always Have Popsicles,1,1,1,1,1,1,1,1,1,1,1
Apple Magic (The Collector's series),1,1,1,1,1,1,1,1,1,1,1
"Ask Lily (Young Women of Faith: Lily Series, Book 5)",1,1,1,1,1,1,1,1,1,1,1
Beyond IBM: Leadership Marketing and Finance for the 1990s,1,1,0,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
Ã?Â?lpiraten.,2,2,1,2,2,2,2,2,2,2,2
Ã?Â?rger mit Produkt X. Roman.,4,4,4,4,4,4,4,4,4,4,4
Ã?Â?sterlich leben.,1,1,1,1,1,1,1,1,1,1,1


In [26]:
df.groupby('Book-Title').count()['Book-Rating'].reset_index()

,Book-Title,Book-Rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [27]:
rating_count = df.groupby('Book-Title').count()['Book-Rating']

In [28]:
average_rating = df.groupby('Book-Title')['Book-Rating'].mean().reset_index()

In [29]:
average_rating

,Book-Title,Book-Rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [30]:
popular_df = average_rating.merge(rating_count, on='Book-Title', how='inner')

In [31]:
popular_df.head()

,Book-Title,Book-Rating_x,Book-Rating_y
0,A Light in the Storm: The Civil War Diary of ...,2.25,4
1,Always Have Popsicles,0.00,1
2,Apple Magic (The Collector's series),0.00,1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00,1
4,Beyond IBM: Leadership Marketing and Finance ...,0.00,1


In [32]:
popular_df.rename(columns={'Book-Rating_x':'avg_ratings', 'Book-Rating_y':'rating_count'},inplace=True)

In [33]:
popular_df.head()

,Book-Title,avg_ratings,rating_count
0,A Light in the Storm: The Civil War Diary of ...,2.25,4
1,Always Have Popsicles,0.00,1
2,Apple Magic (The Collector's series),0.00,1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00,1
4,Beyond IBM: Leadership Marketing and Finance ...,0.00,1


In [34]:
popular_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241071 entries, 0 to 241070
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Book-Title    241071 non-null  object 
 1   avg_ratings   241071 non-null  float64
 2   rating_count  241071 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 5.5+ MB


In [35]:
popular_df = popular_df[popular_df['rating_count']>= 250]

In [36]:
popular_df.sort_values(by='avg_ratings', ascending=False, inplace=True)

In [37]:
popular_df.head()

,Book-Title,avg_ratings,rating_count
80434,Harry Potter and the Prisoner of Azkaban (Book 3),5.852804,428
80422,Harry Potter and the Goblet of Fire (Book 4),5.824289,387
80441,Harry Potter and the Sorcerer's Stone (Book 1),5.737410,278
80426,Harry Potter and the Order of the Phoenix (Boo...,5.501441,347
80414,Harry Potter and the Chamber of Secrets (Book 2),5.183453,556


In [38]:
popular_df.shape

(186, 3)

In [39]:
popular_df.drop_duplicates('Book-Title', inplace=True)

In [40]:
popular_df.shape

(186, 3)

In [41]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [42]:
popular_df = popular_df.merge(books, on='Book-Title').drop_duplicates('Book-Title')[['Book-Title', 'Book-Author', 'Publisher']]

In [43]:
popular_df

,Book-Title,Book-Author,Publisher
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,Scholastic
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,Scholastic
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,Scholastic
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,Scholastic
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,Scholastic
...,...,...,...
716,Vinegar Hill (Oprah's Book Club (Paperback)),A. Manette Ansay,Perennial
717,Whispers,BELVA PLAIN,Dell
727,Presumed Innocent,Scott Turow,Warner Books
733,Isle of Dogs,Patricia Cornwell,Berkley Publishing Group


### Collaborative Filter based recommendation

In [44]:
 rating_provided = df.groupby('User-ID').count()['Book-Rating'] > 200 # fetch serious users , who rate frequently

In [45]:
rating_provided.head().reset_index()

,User-ID,Book-Rating
0,2,False
1,8,False
2,9,False
3,10,False
4,12,False


In [46]:
user_ids= rating_provided[rating_provided].index   # get user ID's
user_ids

Index([   254,   2276,   2766,   2977,   3363,   4017,   4385,   6251,   6323,
         6543,
       ...
       271705, 273979, 274004, 274061, 274301, 274308, 275970, 277427, 277639,
       278418],
      dtype='int64', name='User-ID', length=811)

In [47]:
# rating provided by above selected users
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031136 entries, 0 to 1031135
Data columns (total 12 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   User-ID              1031136 non-null  int64  
 1   Location             1031136 non-null  object 
 2   Age                  753301 non-null   float64
 3   ISBN                 1031136 non-null  object 
 4   Book-Rating          1031136 non-null  int64  
 5   Book-Title           1031136 non-null  object 
 6   Book-Author          1031134 non-null  object 
 7   Year-Of-Publication  1031136 non-null  object 
 8   Publisher            1031134 non-null  object 
 9   Image-URL-S          1031136 non-null  object 
 10  Image-URL-M          1031136 non-null  object 
 11  Image-URL-L          1031132 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 94.4+ MB


In [48]:
df.drop_duplicates(inplace=True)

In [49]:
df_new = df[df['User-ID'].isin(user_ids)]

In [50]:
df_new

,User-ID,Location,Age,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
311,254,"minneapolis, minnesota, usa",24.0,006000438X,0,The Death of Vishnu: A Novel,Manil Suri,2002,Perennial,http://images.amazon.com/images/P/006000438X.0...,http://images.amazon.com/images/P/006000438X.0...,http://images.amazon.com/images/P/006000438X.0...
312,254,"minneapolis, minnesota, usa",24.0,0060013117,0,Night Watch,Terry Pratchett,2002,HarperCollins,http://images.amazon.com/images/P/0060013117.0...,http://images.amazon.com/images/P/0060013117.0...,http://images.amazon.com/images/P/0060013117.0...
313,254,"minneapolis, minnesota, usa",24.0,0060199563,0,Thief of Time,Terry Pratchett,2001,HarperCollins Publishers,http://images.amazon.com/images/P/0060199563.0...,http://images.amazon.com/images/P/0060199563.0...,http://images.amazon.com/images/P/0060199563.0...
314,254,"minneapolis, minnesota, usa",24.0,0060391448,0,Wicked : The Life and Times of the Wicked Witc...,Gregory Maguire,1995,HarperCollins,http://images.amazon.com/images/P/0060391448.0...,http://images.amazon.com/images/P/0060391448.0...,http://images.amazon.com/images/P/0060391448.0...
315,254,"minneapolis, minnesota, usa",24.0,0060502320,7,"I've Got You, Babe",Karen Kendall,2002,Avon,http://images.amazon.com/images/P/0060502320.0...,http://images.amazon.com/images/P/0060502320.0...,http://images.amazon.com/images/P/0060502320.0...
...,...,...,...,...,...,...,...,...,...,...,...,...
1030506,278418,"omaha, nebraska, usa",NaN,1885222971,0,The Lion King: The Pal Patrol (Disney's \Story...,Inc. Staff Disney Enterprises,1997,Advance Publishers LLC,http://images.amazon.com/images/P/1885222971.0...,http://images.amazon.com/images/P/1885222971.0...,http://images.amazon.com/images/P/1885222971.0...
1030507,278418,"omaha, nebraska, usa",NaN,188522298X,0,101 Dalmatians: Proud to Be a Pup,Disney Enterprises Inc,1997,Advance Publishers LLC,http://images.amazon.com/images/P/188522298X.0...,http://images.amazon.com/images/P/188522298X.0...,http://images.amazon.com/images/P/188522298X.0...
1030508,278418,"omaha, nebraska, usa",NaN,1885222998,0,Aladdin: Wishful Thinking (Disney's Storytime ...,Disney Enterprises Inc,1997,Advance Publishers LLC,http://images.amazon.com/images/P/1885222998.0...,http://images.amazon.com/images/P/1885222998.0...,http://images.amazon.com/images/P/1885222998.0...
1030509,278418,"omaha, nebraska, usa",NaN,1892083043,0,Gotcha! Strategy Games for Math and Logic (Hom...,Denise Gaskins,1998,Tabletop Academy Pr,http://images.amazon.com/images/P/1892083043.0...,http://images.amazon.com/images/P/1892083043.0...,http://images.amazon.com/images/P/1892083043.0...


In [51]:
y= df_new.groupby('Book-Title').count()['Book-Rating'] > 50
famous_books=y[y].index

In [52]:
famous_books

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Day Late and a Dollar Short', 'A Fine Balance',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='Book-Title', length=679)

In [53]:
fina_ratings=df_new[df_new['Book-Title'].isin(famous_books)]
fina_ratings

,User-ID,Location,Age,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
319,254,"minneapolis, minnesota, usa",24.0,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...
332,254,"minneapolis, minnesota, usa",24.0,0062502174,0,The Alchemist: A Fable About Following Your Dream,Paulo Coelho,1993,HarperSanFrancisco,http://images.amazon.com/images/P/0062502174.0...,http://images.amazon.com/images/P/0062502174.0...,http://images.amazon.com/images/P/0062502174.0...
337,254,"minneapolis, minnesota, usa",24.0,014028009X,0,Bridget Jones's Diary,Helen Fielding,1999,Penguin Books,http://images.amazon.com/images/P/014028009X.0...,http://images.amazon.com/images/P/014028009X.0...,http://images.amazon.com/images/P/014028009X.0...
338,254,"minneapolis, minnesota, usa",24.0,0140298479,0,Bridget Jones: The Edge of Reason,Helen Fielding,2001,Penguin Books,http://images.amazon.com/images/P/0140298479.0...,http://images.amazon.com/images/P/0140298479.0...,http://images.amazon.com/images/P/0140298479.0...
342,254,"minneapolis, minnesota, usa",24.0,014100018X,0,Chocolat,Joanne Harris,2000,Penguin Books,http://images.amazon.com/images/P/014100018X.0...,http://images.amazon.com/images/P/014100018X.0...,http://images.amazon.com/images/P/014100018X.0...
...,...,...,...,...,...,...,...,...,...,...,...,...
1029815,278418,"omaha, nebraska, usa",NaN,0786817070,0,"Artemis Fowl (Artemis Fowl, Book 1)",Eoin Colfer,2002,Miramax Kids,http://images.amazon.com/images/P/0786817070.0...,http://images.amazon.com/images/P/0786817070.0...,http://images.amazon.com/images/P/0786817070.0...
1029827,278418,"omaha, nebraska, usa",NaN,0786889020,0,Don't Stand Too Close to a Naked Man,Tim Allen,1995,Hyperion,http://images.amazon.com/images/P/0786889020.0...,http://images.amazon.com/images/P/0786889020.0...,http://images.amazon.com/images/P/0786889020.0...
1029857,278418,"omaha, nebraska, usa",NaN,080410526X,0,All I Really Need to Know,ROBERT FULGHUM,1989,Ivy Books,http://images.amazon.com/images/P/080410526X.0...,http://images.amazon.com/images/P/080410526X.0...,http://images.amazon.com/images/P/080410526X.0...
1030026,278418,"omaha, nebraska, usa",NaN,0821759744,0,Dangerous,Becky Barker,1997,Kensington Publishing Corporation,http://images.amazon.com/images/P/0821759744.0...,http://images.amazon.com/images/P/0821759744.0...,http://images.amazon.com/images/P/0821759744.0...


In [54]:
fina_ratings.drop_duplicates()

,User-ID,Location,Age,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
319,254,"minneapolis, minnesota, usa",24.0,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...
332,254,"minneapolis, minnesota, usa",24.0,0062502174,0,The Alchemist: A Fable About Following Your Dream,Paulo Coelho,1993,HarperSanFrancisco,http://images.amazon.com/images/P/0062502174.0...,http://images.amazon.com/images/P/0062502174.0...,http://images.amazon.com/images/P/0062502174.0...
337,254,"minneapolis, minnesota, usa",24.0,014028009X,0,Bridget Jones's Diary,Helen Fielding,1999,Penguin Books,http://images.amazon.com/images/P/014028009X.0...,http://images.amazon.com/images/P/014028009X.0...,http://images.amazon.com/images/P/014028009X.0...
338,254,"minneapolis, minnesota, usa",24.0,0140298479,0,Bridget Jones: The Edge of Reason,Helen Fielding,2001,Penguin Books,http://images.amazon.com/images/P/0140298479.0...,http://images.amazon.com/images/P/0140298479.0...,http://images.amazon.com/images/P/0140298479.0...
342,254,"minneapolis, minnesota, usa",24.0,014100018X,0,Chocolat,Joanne Harris,2000,Penguin Books,http://images.amazon.com/images/P/014100018X.0...,http://images.amazon.com/images/P/014100018X.0...,http://images.amazon.com/images/P/014100018X.0...
...,...,...,...,...,...,...,...,...,...,...,...,...
1029815,278418,"omaha, nebraska, usa",NaN,0786817070,0,"Artemis Fowl (Artemis Fowl, Book 1)",Eoin Colfer,2002,Miramax Kids,http://images.amazon.com/images/P/0786817070.0...,http://images.amazon.com/images/P/0786817070.0...,http://images.amazon.com/images/P/0786817070.0...
1029827,278418,"omaha, nebraska, usa",NaN,0786889020,0,Don't Stand Too Close to a Naked Man,Tim Allen,1995,Hyperion,http://images.amazon.com/images/P/0786889020.0...,http://images.amazon.com/images/P/0786889020.0...,http://images.amazon.com/images/P/0786889020.0...
1029857,278418,"omaha, nebraska, usa",NaN,080410526X,0,All I Really Need to Know,ROBERT FULGHUM,1989,Ivy Books,http://images.amazon.com/images/P/080410526X.0...,http://images.amazon.com/images/P/080410526X.0...,http://images.amazon.com/images/P/080410526X.0...
1030026,278418,"omaha, nebraska, usa",NaN,0821759744,0,Dangerous,Becky Barker,1997,Kensington Publishing Corporation,http://images.amazon.com/images/P/0821759744.0...,http://images.amazon.com/images/P/0821759744.0...,http://images.amazon.com/images/P/0821759744.0...


In [55]:
my_table= fina_ratings.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')

In [56]:
my_table.fillna(0, inplace=True)

In [57]:
my_table  # collabration table is ready

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
from sklearn.metrics.pairwise import cosine_similarity

In [59]:
similarity_score= cosine_similarity(my_table)
similarity_score

array([[1.        , 0.10255025, 0.01220856, ..., 0.12110367, 0.07347567,
        0.04316046],
       [0.10255025, 1.        , 0.2364573 , ..., 0.07446129, 0.16773875,
        0.14263397],
       [0.01220856, 0.2364573 , 1.        , ..., 0.04558758, 0.04938579,
        0.10796119],
       ...,
       [0.12110367, 0.07446129, 0.04558758, ..., 1.        , 0.07085128,
        0.0196177 ],
       [0.07347567, 0.16773875, 0.04938579, ..., 0.07085128, 1.        ,
        0.10602962],
       [0.04316046, 0.14263397, 0.10796119, ..., 0.0196177 , 0.10602962,
        1.        ]])

In [60]:
similarity_score.shape

(679, 679)

In [61]:
#index calculation
index123 =np.where(my_table.index =='Zoya')[0][0]
index123

677

In [62]:
sorted(list(enumerate(similarity_score[index123])),key= lambda l: l[1], reverse=True)[1:6] # sort on the basis of second col, i.e. rating col, 
#pick top 5 excluding the index123 item

[(187, 0.6188332046777835),
 (409, 0.5423917439488084),
 (278, 0.4536936187776783),
 (168, 0.43416881684252095),
 (189, 0.43104197190280946)]

In [63]:
def recommend(book_name):
    # fetch book index
    book_index=np.where(my_table.index == book_name)[0][0]
    print(f'book index - {book_index}')    
    # find similar score book
    similar_item_list = sorted(list(enumerate(similarity_score[book_index])),key= lambda p: p[1], reverse=True)[1:6]    
    for i in similar_item_list:
        print(my_table.index[i[0]])



In [64]:
recommend('Harry Potter and the Prisoner of Azkaban (Book 3)')

book index - 221
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Order of the Phoenix (Book 5)
Harry Potter and the Sorcerer's Stone (Book 1)
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
